<a href="https://colab.research.google.com/github/forfourr/DL_test/blob/main/Keras_ex_01_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import mnist

Review data(TEXT) -> encoding
- onehot encoding
    the: [1,0,0]
    it: [0,1,0]
- Bow(Bag of Words)

In [ ]:
# import data
from keras.datasets import imdb
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=1000)

In [ ]:
#단어의 종류는 1000개이고 25000가지의 데이터 가져옴
train_data.shape

이미 one-hot 인코딩 프로세싱 됨. data->숫자, label=0/1(싫다/좋다)

In [ ]:
#전체 단어가 몇개인지 확인
cnt=0
for sequence in train_data:
    cnt+=1
print("총 단어의 개수: ",cnt)

In [ ]:
max([max(squence) for squence in train_data])      
#for문의 결과의 sequence의 max, 그 max중에서 또 max


- 적당한 크기의 배열로 만드는 과정 

    25000건의 데이터를 넘기면 len(sequence)=25000
    
    25000개 요소를 가진 10000차원 배열
    -> zeros(25000,10000)

In [ ]:
#data 준비
import numpy as np
def vectorize_sequences(sequences, dimension=10000):     #10000의 데이터가 있음
    result = np.zeros((len(sequences),dimension))         #배열의 크기는 sequences의 길이의 디멘션
    print(result.shape)
            #np.zeros(25000,10000)                      #zeros -> 모든원소 0 
    for i, sequence in enumerate(sequences):
        result[i, sequence] =1                          #특정 인덱스의 위치를 1로 만듬
    return result

enumerate : 각 배열에 index 숫자 순서대로 붙여줌

i: 순서대로 붙인 index


sequence: data값

-> zeros고차원 배열에서 단어가 위치한 곳에 1을 넣음

In [ ]:
#데이터를 벡터로 변환
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
# label data ->float
# x_train이 float이기 때문에
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
#신경망 제작
from keras import models
from keras import layers

model = models.Sequential()

#Add layer 3
model.add(layers.Dense(16, activation='relu',       #input layer, unit 16
                       input_shape=(10000,)))     
model.add(layers.Dense(16, activation='relu'))      #hidden layer
model.add(layers.Dense(1, activation='sigmoid'))    #output layer, 이진분류이기 때문에 unit=1

#Loss function, optimizer, metrics 결정
model.compile(optimizer='rmsprop',
              loss= 'binary_crossentropy',
              metrics= ['accuracy'])

- 검증용 데이터 나누기

25000(x_train) = 10000(val) + 15000(partical)

partical로 먼저 training

In [ ]:
#검증용 데이터
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
#학습진행 내용을 history에 저장시켜 나중에 시각화 해서 봄
history = model.fit(partial_x_train, 
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

val_loss, val_accuracy는 우리가 train하지 않은 나머지 값으로 했을 때의 결과

val_loss는 낮아지다가 높아지는 것을 볼 수 있다

https://colab.research.google.com/drive/1bwaqbwJu91Z4Fy35xXWN8YYmh1sRtspP?usp=sharing#scrollTo=7Ncy598QL1TC

In [ ]:
# 시각화
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc)+1)

#loss
plt.plot(epochs,loss, label='loss')
plt.plot(epochs,val_loss, 'r', label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
#accuary
plt.plot(epochs,history.history['accuracy'], label='accuracy')
plt.plot(epochs,history.history['val_accuracy'], 'r',label='val_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
model.predict(x_test)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('loss={0}, accuracy={1}'.format(test_loss, test_acc))